In [2]:
import geopandas as gpd
from gadm import GADMDownloader

import rasterio
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import os
import datetime
import glob
import time

In [7]:
import pandas as pd
import numpy as np
import requests
import time
import os
import re
import warnings
import datetime 
import tempfile
import subprocess
import glob
import shutil
from itertools import product
import geopandas as gpd
from rasterstats import zonal_stats
import h5py
import rasterio
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.merge import merge
from rasterio.transform import from_origin


In [13]:
from utils import (cross_df,
                month_start_day_to_month,
                pad2,
                pad3,
                file_to_raster,
                read_bm_csv,
                create_dataset_name_df,
                download_raster,
                define_variable,
                define_date_name,
                bm_extract_i,
                bm_raster_i)

In [14]:
downloader = GADMDownloader(version="4.0")

country_name = "Ghana"
ad_level = 1
roi_sf = downloader.get_shape_data_by_country_name(country_name=country_name, ad_level=ad_level)

In [15]:
product_id = 'VNP46A4'
date = 2021

if os.path.exists("/Users/robmarty/Desktop/bearer_bm.csv"):
    bearer = pd.read_csv("/Users/robmarty/Desktop/bearer_bm.csv")['token'][0]
else:
    bearer == "BEARER TOKEN HERE"

In [24]:

variable=None
quality_flag_rm = [255]
check_all_tiles_exist = True
output_location_type="tempfile"
file_dir=None
file_prefix=""
file_skip_if_exists=True
quiet = False
 
    
#### Make directory to put temporary files into
temp_main_dir = tempfile.gettempdir()

current_time_millis = int(round(time.time() * 1000))
temp_dir = os.path.join(temp_main_dir, "bm_raster_temp_" + str(current_time_millis))

os.makedirs(temp_dir)

#### Define NTL Variable
variable = define_variable(variable, product_id)

#### Ensure quality_flag_rm is a list
if type(quality_flag_rm) is not list:
    quality_flag_rm = [quality_flag_rm]

#### Ensure date is a list of strings
if type(date) is not list:
    date = [date]

date = [str(item) for item in date]

if file_dir is None:
    file_dir = os.getcwd()


In [31]:
date_i = "2021"

date_name_i = define_date_name(date_i, product_id)



,TileID,geometry
0,h00v00,"POLYGON ((-180.00000 90.00012, -170.00000 90.0..."
1,h00v01,"POLYGON ((-180.00000 80.00012, -170.00000 80.0..."
2,h00v02,"POLYGON ((-180.00000 70.00012, -170.00000 70.0..."
3,h00v03,"POLYGON ((-180.00000 60.00012, -170.00000 60.0..."
4,h00v04,"POLYGON ((-180.00000 50.00012, -170.00000 50.0..."
...,...,...
643,h35v13,"POLYGON ((170.00012 -40.00000, 180.00012 -40.0..."
644,h35v14,"POLYGON ((170.00012 -50.00000, 180.00012 -50.0..."
645,h35v15,"POLYGON ((170.00012 -60.00000, 180.00012 -60.0..."
646,h35v16,"POLYGON ((170.00012 -70.00000, 180.00012 -70.0..."


In [ ]:


try:
    #### Create raster for each date
    raster_path_list = []    
    for date_i in date:

        date_name_i = define_date_name(date_i, product_id)

        raster_path_i = bm_raster_i(roi_sf, product_id, 
                                    date_i, bearer, 
                                    variable, quality_flag_rm, check_all_tiles_exist, quiet, temp_dir)
        raster_path_list.append(raster_path_i) 

    #### Stack Rasters
    # Read the first file to get the dimensions and metadata
    with rasterio.open(raster_path_list[0]) as src:
        width = src.width
        height = src.height
        count = len(raster_path_list)
        crs = src.crs
        transform = src.transform
        dtype = src.dtypes[0]

        # Create an empty numpy array to store the raster data
        data = np.zeros((count, height, width), dtype=dtype)

        # Read data from each file and store it in the numpy array
        for i, file_path in enumerate(raster_path_list):
            with rasterio.open(file_path) as src:
                data[i] = src.read(1)  

    # Create a new raster file and write the data
    timestamp = str(int(time.time()))
    tmp_raster_file_name = product_id + "_" + date[0].replace("-", "_") + "_" + timestamp + ".tif"

    with rasterio.open(os.path.join(temp_dir, 
                                    tmp_raster_file_name), 
                       'w', driver='GTiff', width=width, height=height,
                       count=count, dtype=dtype, crs=crs, transform=transform) as dst:
        for i in range(count):
            dst.write(data[i], i+1) 

    # Move from black marble temp folder to main temp folder
    shutil.move(os.path.join(temp_dir, tmp_raster_file_name), 
                temp_main_dir) 

    r_out = rasterio.open(os.path.join(temp_main_dir, tmp_raster_file_name))

except:
    # Delete temp files used to make raster
    shutil.rmtree(temp_dir, ignore_errors=True)

    if quiet == False:
        print("Skipping " + str(date_i) + " due to error. Data may not be available.\n")
    r_out = None

# Delete temp files used to make raster
shutil.rmtree(temp_dir, ignore_errors=True)
